In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as LA
import os

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, MaxPooling2D, Dense, Conv2D, Flatten
from tensorflow.keras import optimizers
import tensorflow.keras.preprocessing.image as img
from sklearn.metrics import r2_score

In [3]:
def processData():
    dataset, labels = [], []
    dirPath = '../data/HW10/HW10train/'

    for im in os.listdir(dirPath):
        label = float(im[:2])
        data = img.load_img(dirPath+im)
        data = img.img_to_array(data)
        
        dataset.append(data)
        labels.append(label)

    print("Training set: ", np.array(dataset).shape)
    print("Training labels: ", np.array(labels).shape)
    
    np.save("../data/HW10/Train.data", np.array(dataset))
    np.save("../data/HW10/Train.labels", np.array(labels))
    
    dataset, labels = [], []
    dirPath = '../data/HW10/HW10test/'

    for im in os.listdir(dirPath):
        label = float(im[:2])
        data = img.load_img(dirPath+im)
        data = img.img_to_array(data)
        
        dataset.append(data)
        labels.append(label)
 
    print("Testing set: ", np.array(dataset).shape)
    print("Testing labels: ", np.array(labels).shape)
    
    np.save("../data/HW10/Test.data", np.array(dataset))
    np.save("../data/HW10/Test.labels", np.array(labels))
    

#processData()

In [4]:
xtrain, ytrain = np.load('../data/HW10/Train.data.npy'), np.load('../data/HW10/Train.labels.npy')
xtest, ytest = np.load('../data/HW10/Test.data.npy'), np.load('../data/HW10/Test.labels.npy')

print('xtrain: ', xtrain.shape, '\n ytrain: ', ytrain.shape)
print('xtest: ', xtest.shape, '\n ytest: ', ytest.shape)

print('xtrain: ', xtrain.dtype, '\n ytrain: ', ytrain.dtype)
print('xtest: ', xtest.dtype, '\n ytest: ', ytest.dtype)

xtrain:  (18059, 64, 64, 3) 
 ytrain:  (18059,)
xtest:  (2261, 64, 64, 3) 
 ytest:  (2261,)
xtrain:  float32 
 ytrain:  float64
xtest:  float32 
 ytest:  float64


In [5]:
CNN = Sequential()

# First Layer
CNN.add(Conv2D(16, (5, 5), strides=1, input_shape=(64, 64, 3)))

# Second Layer
CNN.add(Conv2D(16, (5, 5), strides=1))

# Third Layer
CNN.add(Conv2D(32, (5, 5), strides=1))
CNN.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Fourth Layer
CNN.add(Conv2D(32, (5, 5), strides=1))
CNN.add(Activation('relu'))

# Final Layer
CNN.add(Conv2D(1, (1, 1), strides=1))
CNN.add(Flatten())
CNN.add(Dense(1))


In [6]:
optim = optimizers.Adam(learning_rate=0.0001)
CNN.compile(optimizer=optim, loss='mean_squared_error')
CNN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 16)        1216      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 16)        6416      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 32)        12832     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 32)        25632     
_________________________________________________________________
activation (Activation)      (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 22, 1)         3

In [7]:
batches = [32, 64, 128, 256, 512]

for b in batches:
    print("\n=-=-=-=-=-=-=-=-= Batch Size ", b, " =-=-=-=-=-=-=-=-=")
   
    CNN.fit(xtrain, ytrain, epochs=20, batch_size = b, verbose=2)
    
    preds = CNN.predict(xtrain)
    score = r2_score(ytrain, preds)
    print("Train R2: ", score)
    
    preds = CNN.predict(xtest)
    score = r2_score(ytest, preds)
    print("Test R2: ", score)
    print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n")
    


=-=-=-=-=-=-=-=-= Batch Size  32  =-=-=-=-=-=-=-=-=
Train on 18059 samples
Epoch 1/20
18059/18059 - 5s - loss: 800.5002
Epoch 2/20
18059/18059 - 3s - loss: 360.7324
Epoch 3/20
18059/18059 - 3s - loss: 272.0849
Epoch 4/20
18059/18059 - 3s - loss: 216.4255
Epoch 5/20
18059/18059 - 3s - loss: 178.1774
Epoch 6/20
18059/18059 - 3s - loss: 153.5256
Epoch 7/20
18059/18059 - 3s - loss: 136.2219
Epoch 8/20
18059/18059 - 3s - loss: 123.2112
Epoch 9/20
18059/18059 - 3s - loss: 116.9183
Epoch 10/20
18059/18059 - 3s - loss: 106.8231
Epoch 11/20
18059/18059 - 3s - loss: 101.7645
Epoch 12/20
18059/18059 - 3s - loss: 96.2141
Epoch 13/20
18059/18059 - 3s - loss: 89.9674
Epoch 14/20
18059/18059 - 3s - loss: 87.7351
Epoch 15/20
18059/18059 - 3s - loss: 83.9449
Epoch 16/20
18059/18059 - 3s - loss: 79.1601
Epoch 17/20
18059/18059 - 3s - loss: 74.8855
Epoch 18/20
18059/18059 - 3s - loss: 72.6592
Epoch 19/20
18059/18059 - 3s - loss: 70.3451
Epoch 20/20
18059/18059 - 3s - loss: 66.3218
Train R2:  0.871039955

In [8]:
#preds = CNN.predict(xtrain)
#r2_score(ytrain, preds)

In [9]:
#preds = CNN.predict(xtest)
#r2_score(ytest, preds)

In [10]:
#for i in CNN.predict(xtrain):
#    print(i)